## Imports & Setup

In [4]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

# Config
RAW_DIR = os.path.join(os.getcwd(), "data", "raw_full")
PROCESSED_DIR = os.path.join(os.getcwd(), "data", "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Threshold: Drop columns with > 50% missing values
MISSING_THRESHOLD = 0.5 

print(f"📂 Reading from: {RAW_DIR}")
print(f"🎯 Saving to: {PROCESSED_DIR}")

📂 Reading from: /workspaces/MAP_Hackathon_2026_G4/src/data/raw_full
🎯 Saving to: /workspaces/MAP_Hackathon_2026_G4/src/data/processed


## Load All Data

In [5]:
# 1. Get all file paths
all_files = glob.glob(os.path.join(RAW_DIR, "*.xpt")) + glob.glob(os.path.join(RAW_DIR, "*.XPT"))
print(f"🔍 Found {len(all_files)} files.")

# 2. Find DEMO file (Anchor dataset)
demo_path = next((f for f in all_files if "DEMO" in os.path.basename(f).upper()), None)
if not demo_path:
    raise FileNotFoundError("CRITICAL: DEMO file not found! Cannot start merging.")

print(f"⚓ Anchor File: {os.path.basename(demo_path)}")
df_master = pd.read_sas(demo_path)
df_master['SEQN'] = df_master['SEQN'].astype(int)
base_seqns = set(df_master['SEQN'])  # For rapid screen

print(f"   Initial Shape: {df_master.shape}")
print(f"   Memory Usage: {df_master.memory_usage().sum() / 1024**2:.2f} MB")

🔍 Found 131 files.
⚓ Anchor File: DEMO_J.xpt
   Initial Shape: (9254, 46)
   Memory Usage: 3.25 MB


## Memory-Optimized Iterative Merge

In [6]:
count = 0
dropped_vars_count = 0

print("🚀 Starting iterative merge...")

for f in all_files:
    if f == demo_path: continue
    
    try:
        # A. Read file
        df_temp = pd.read_sas(f)
        
        # B. Check for SEQN (Primary Key)
        if 'SEQN' not in df_temp.columns:
            continue
        
        df_temp['SEQN'] = df_temp['SEQN'].astype(int)
        
        # C. [Optimization] Row Filtering
        # Only keep rows that exist in the base DEMO dataset.
        # This removes data for participants not in the main demographic group (e.g., children if excluded).
        df_temp = df_temp[df_temp['SEQN'].isin(base_seqns)]
        
        if df_temp.empty:
            continue

        # D. [Optimization] Pre-merge Column Filtering
        # Calculate missing rate BEFORE merging. If a column is mostly empty, don't bring it in.
        cols_to_keep = ['SEQN']
        for col in df_temp.columns:
            if col == 'SEQN': continue
            
            # Calculate missing percentage for this column
            missing_rate = df_temp[col].isnull().mean()
            
            if missing_rate <= MISSING_THRESHOLD:
                cols_to_keep.append(col)
            else:
                dropped_vars_count += 1
        
        # Keep only the useful columns
        df_temp = df_temp[cols_to_keep]
        
        # If no columns remain (other than SEQN), skip this file
        if len(df_temp.columns) <= 1:
            continue

        # E. Merge into Master DataFrame
        df_master = pd.merge(df_master, df_temp, on='SEQN', how='left')
        
        count += 1
        
        # F. [Optimization] Garbage Collection
        # Explicitly delete the temporary dataframe and run garbage collection to free up RAM.
        del df_temp
        if count % 10 == 0:
            print(f"   ... Merged {count} files. Master Shape: {df_master.shape}")
            gc.collect()
            
    except Exception as e:
        print(f"⚠️ Error processing {os.path.basename(f)}: {e}")

🚀 Starting iterative merge...
   ... Merged 10 files. Master Shape: (17529, 758)
⚠️ Error processing FASTQX_J.xpt: name 'gc' is not defined


: 

## Final Save

In [ ]:
print("\n" + "="*40)
print(f"🎉 Merge Complete!")
print(f"📊 Final Shape: {df_master.shape}")
print(f"🗑️ Total Variables Dropped (Pre-filter): {dropped_vars_count}")

# Save as CSV
csv_path = os.path.join(PROCESSED_DIR, "nhanes_2017_2018_filtered.csv")
df_master.to_csv(csv_path, index=False)
print(f"💾 Saved CSV to: {csv_path}")

# Save as Pickle (Recommended for Python workflows - reads/writes much faster)
pkl_path = os.path.join(PROCESSED_DIR, "nhanes_2017_2018_filtered.pkl")
df_master.to_pickle(pkl_path)
print(f"💾 Saved Pickle to: {pkl_path}")

## Calculate Missing Rates (Memory Safe)

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import gc  # Garbage Collection

# === Configuration ===
# Path to your raw XPT files
RAW_DIR = os.path.join(os.getcwd(), "data", "raw_full")
MISSING_THRESHOLD = 0.5  # 50% threshold

print(f"📂 Scanning files in: {RAW_DIR}")
all_files = glob.glob(os.path.join(RAW_DIR, "*.xpt")) + glob.glob(os.path.join(RAW_DIR, "*.XPT"))
print(f"🔍 Found {len(all_files)} files. Calculating missing rates iteratively...")

var_missing_rates = {}

for i, f in enumerate(all_files):
    try:
        # Read one file at a time
        df_temp = pd.read_sas(f)
        
        # Calculate missing rate for each variable in this file
        # This returns a Series like: {'SEQN': 0.0, 'LBXGLU': 0.1, ...}
        rates = df_temp.isnull().mean()
        
        # Store in our master dictionary
        # Note: If a variable appears in multiple files (like SEQN), this will overwrite with the latest.
        # Since variables are mostly unique to specific tables, this is fine for a general overview.
        var_missing_rates.update(rates.to_dict())
        
        # Free memory immediately
        del df_temp
        gc.collect()
        
        # Progress indicator
        if (i + 1) % 20 == 0:
            print(f"   Processed {i + 1}/{len(all_files)} files...")
            
    except Exception as e:
        print(f"⚠️ Error reading {os.path.basename(f)}: {e}")

# Convert dictionary to Series for your plotting code
missing_series = pd.Series(var_missing_rates)

print(f"\n✅ Analysis Complete!")
print(f"   Total Variables Analyzed: {len(missing_series)}")
print(f"   Variables to Keep (< {MISSING_THRESHOLD*100}% missing): {len(missing_series[missing_series <= MISSING_THRESHOLD])}")
print(f"   Variables to Drop (> {MISSING_THRESHOLD*100}% missing): {len(missing_series[missing_series > MISSING_THRESHOLD])}")

## Visualization

In [ ]:
# what kept vs dropped
plt.figure(figsize=(10, 5))
plt.hist(missing_series * 100, bins=20, color='skyblue', edgecolor='black')
plt.axvline(MISSING_THRESHOLD * 100, color='red', linestyle='--', label=f'Cutoff ({MISSING_THRESHOLD*100}%)')
plt.title('Distribution of Missing Data % Across All Variables')
plt.xlabel('% Missing')
plt.ylabel('Number of Variables')
plt.legend()
plt.show()

## Filter by Missing Values

In [ ]:
print("🧹 Starting Filtration...")

# 1. Calculate missing rates
missing_series = df_master.isnull().mean()

# 2. Identify keep vs drop
keep_cols = missing_series[missing_series <= MISSING_THRESHOLD].index
drop_cols = missing_series[missing_series > MISSING_THRESHOLD].index

print(f"   Total Variables: {len(missing_series)}")
print(f"   Keep (<= {MISSING_THRESHOLD*100}% missing): {len(keep_cols)}")
print(f"   Drop (> {MISSING_THRESHOLD*100}% missing): {len(drop_cols)}")

# 3. Create filtered DataFrame
df_clean = df_master[keep_cols].copy()

# 4. Save
save_path = os.path.join(PROCESSED_DIR, "nhanes_2017_2018_filtered.csv")
# Use pickle for faster IO in Python, or CSV for compatibility
df_clean.to_csv(save_path, index=False)
df_clean.to_pickle(os.path.join(PROCESSED_DIR, "nhanes_2017_2018_filtered.pkl"))

print(f"\n🎉 Saved clean data to: {save_path}")
print(f"📊 Final Clean Shape: {df_clean.shape}")

## Visualize Missing Data

In [ ]:
# Let's see what we kept vs dropped
plt.figure(figsize=(10, 5))
plt.hist(missing_series * 100, bins=20, color='skyblue', edgecolor='black')
plt.axvline(MISSING_THRESHOLD * 100, color='red', linestyle='--', label=f'Cutoff ({MISSING_THRESHOLD*100}%)')
plt.title('Distribution of Missing Data % Across All Variables')
plt.xlabel('% Missing')
plt.ylabel('Number of Variables')
plt.legend()
plt.show()